# SQL QA: Road Accident Data

link to Excel Dashboard : https://drive.google.com/drive/folders/1wytKJj4COHSZxhVm2_Ryebo2kDeC1rDk?usp=drive_link

In [70]:
import csv, sqlite3

con = sqlite3.connect("RoadAccidents.db")
cur = con.cursor()

In [71]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [72]:
%sql sqlite:///RoadAccidents.db

## Load CSV

In [73]:
import pandas
df = pandas.read_csv('Road Accident Data.csv')
df.to_sql("road_accident", con, if_exists='replace', index=False, method="multi", chunksize=300)

307973

In [74]:
%sql SELECT name FROM sqlite_master WHERE type='table'

 * sqlite:///RoadAccidents.db
Done.


name
road_accident


## FIRST KPI

### 1. Total Casualties

In [75]:
%%sql
SELECT SUM(Number_of_Casualties) AS total_casualties
FROM road_accident

 * sqlite:///RoadAccidents.db
Done.


total_casualties
417883


### 2. Sum of Current year Casualties

In [76]:
%%sql
SELECT SUM(Number_of_Casualties) AS cy_casualties
FROM road_accident
WHERE Year = 2022

 * sqlite:///RoadAccidents.db
Done.


cy_casualties
195737


### 3. Distinct current year Casualties count

In [77]:
%%sql
SELECT COUNT(DISTINCT Accident_Index) as cy_accidents
FROM road_accident
WHERE Year = 2022

 * sqlite:///RoadAccidents.db
Done.


cy_accidents
144419


### 4. Fatal Accidents sum

In [78]:
%%sql
SELECT SUM(Number_of_Casualties) as cy_fatal_accidents
FROM road_accident
WHERE Year = 2022 and Accident_Severity = 'Fatal'

 * sqlite:///RoadAccidents.db
Done.


cy_fatal_accidents
2855


### 5. Fatal Accidents percentage

In [79]:
%%sql
SELECT CAST(SUM(Number_of_Casualties) AS DECIMAL(10,2)) * 100 /
(SELECT CAST(SUM(Number_of_Casualties) AS DECIMAL(10,2)) FROM road_accident) AS percentage_total_fatal
FROM road_accident
WHERE Accident_Severity = 'Fatal'

 * sqlite:///RoadAccidents.db
Done.


percentage_total_fatal
1


### 6. Serious Accidents sum

In [80]:
%%sql
SELECT SUM(Number_of_Casualties) as cy_serious_accidents
FROM road_accident
WHERE Year = 2022 and Accident_Severity = 'Serious'


 * sqlite:///RoadAccidents.db
Done.


cy_serious_accidents
27045


### 7. Serious Accidents percentage

In [81]:
%%sql
SELECT CAST(SUM(Number_of_Casualties) AS DECIMAL(10,2)) * 100 /
(SELECT CAST(SUM(Number_of_Casualties) AS DECIMAL(10,2)) FROM road_accident) AS percentage_total_serious
FROM road_accident
WHERE Accident_Severity = 'Serious'

 * sqlite:///RoadAccidents.db
Done.


percentage_total_serious
14


### 8. Slight Accidents sum

In [82]:
%%sql
SELECT SUM(Number_of_Casualties) as cy_slight_accidents
FROM road_accident
WHERE Year = 2022 and Accident_Severity = 'Slight'

 * sqlite:///RoadAccidents.db
Done.


cy_slight_accidents
165837


### 9. Slight Accidents percentage

In [83]:
%%sql
SELECT CAST(SUM(Number_of_Casualties) AS DECIMAL(10,2)) * 100 /
(SELECT CAST(SUM(Number_of_Casualties) AS DECIMAL(10,2)) FROM road_accident) AS percentage_total_slight
FROM road_accident
WHERE Accident_Severity = 'Slight'

 * sqlite:///RoadAccidents.db
Done.


percentage_total_slight
84


### 10. Accidents by Car

In [84]:
%%sql
SELECT
	CASE
		WHEN Vehicle_Type IN ('Car', 'Taxi/Private hire car') THEN 'Cars'
		ELSE 'Other'
	END AS cars,
	SUM(Number_of_Casualties)
    as car_casualties
FROM road_accident
WHERE Year = 2022
GROUP BY
	CASE
		WHEN Vehicle_Type IN ('Car', 'Taxi/Private hire car') THEN 'Cars'
		ELSE 'Other'
	END

 * sqlite:///RoadAccidents.db
Done.


cars,car_casualties
Cars,155804
Other,39933


### 11. Accidents by Car (Percentage)

In [85]:
%%sql
SELECT
	CASE
		WHEN Vehicle_Type IN ('Car', 'Taxi/Private hire car') THEN 'Cars'
		ELSE 'Other'
	END AS cars,
	CAST(CAST(SUM(Number_of_Casualties) AS DECIMAL(10,2)) *100 / 
	(SELECT CAST(SUM(Number_of_Casualties) AS DECIMAL(10,2)) FROM road_accident WHERE Year = 2022) AS DECIMAL(10,2))
	as car_pertentage_casualties
FROM road_accident
WHERE Year = 2022
GROUP BY
	CASE
		WHEN Vehicle_Type IN ('Car', 'Taxi/Private hire car') THEN 'Cars'
		ELSE 'Other'
	END

 * sqlite:///RoadAccidents.db


Done.


cars,car_pertentage_casualties
Cars,79
Other,20


## SECOND KPI

In [86]:
%%sql
SELECT
	CASE
		WHEN Vehicle_Type IN ('Agricultural vehicle') THEN 'Agricultural'
		WHEN Vehicle_Type IN ('Car', 'Taxi/Private hire car') THEN 'Cars'
		WHEN Vehicle_Type IN ('Motorcycle over 500cc', 'Motorcycle 125cc and under', 'Motorcycle 50cc and under', 'Motorcycle over 125cc and up to 500cc') THEN 'Bike'
		WHEN Vehicle_Type IN ('Bus or coach (17 or more pass seats)', 'Minibus (8 - 16 passenger seats)') THEN 'Bus'
		WHEN Vehicle_Type IN ('Goods 7.5 tonnes mgw and over', 'Goods over 3.5t. and under 7.5t', 'Van / Goods 3.5 tonnes mgw or under') THEN 'Van'
		ELSE 'Other'
	END AS vehicle_group,
	SUM(Number_of_Casualties) as cy_casualties
FROM road_accident
WHERE Year = 2022
GROUP BY
	CASE
		WHEN Vehicle_Type IN ('Agricultural vehicle') THEN 'Agricultural'
		WHEN Vehicle_Type IN ('Car', 'Taxi/Private hire car') THEN 'Cars'
		WHEN Vehicle_Type IN ('Motorcycle over 500cc', 'Motorcycle 125cc and under', 'Motorcycle 50cc and under', 'Motorcycle over 125cc and up to 500cc') THEN 'Bike'
		WHEN Vehicle_Type IN ('Bus or coach (17 or more pass seats)', 'Minibus (8 - 16 passenger seats)') THEN 'Bus'
		WHEN Vehicle_Type IN ('Goods 7.5 tonnes mgw and over', 'Goods over 3.5t. and under 7.5t', 'Van / Goods 3.5 tonnes mgw or under') THEN 'Van'
		ELSE 'Other'
	END


 * sqlite:///RoadAccidents.db
Done.


vehicle_group,cy_casualties
Agricultural,399
Bike,15579
Bus,6573
Cars,155804
Other,1477
Van,15905


## Casualties by year: monthly trend

### 1. Current year

In [87]:
%%sql 
SELECT Month, SUM(Number_of_Casualties) AS cy_casualties
 FROM road_accident
 WHERE Year = '2022'
 GROUP BY Month

 * sqlite:///RoadAccidents.db
Done.


Month,cy_casualties
Apr,15767
Aug,16796
Dec,13200
Feb,14804
Jan,13163
Jul,17201
Jun,17230
Mar,16575
May,16775
Nov,18439


### 2. Previous year 

In [98]:
%%sql 
SELECT Month, SUM(Number_of_Casualties) AS py_casualties
 FROM road_accident
 WHERE Year = '2021'
 GROUP BY Month

 * sqlite:///RoadAccidents.db
Done.


Month,py_casualties
Apr,17335
Aug,18797
Dec,18576
Feb,14648
Jan,18173
Jul,19682
Jun,18728
Mar,17815
May,18852
Nov,20975


## Casualties by Road type

In [89]:
%%sql
 SELECT Road_Type, SUM(Number_of_Casualties) AS cy_casualties
 FROM road_accident
 WHERE Year = '2022'
 GROUP BY Road_Type


 * sqlite:///RoadAccidents.db
Done.


Road_Type,cy_casualties
Dual carriageway,31912
One way street,3499
Roundabout,12683
Single carriageway,144653
Slip road,2990


## Casualties by Rural or Urban areas

### 1. Sum

In [90]:
%%sql
SELECT Urban_or_Rural_Area, SUM(Number_of_Casualties) AS cy_casualties
 FROM road_accident
 WHERE Year = '2022'
 GROUP BY Urban_or_Rural_Area

 * sqlite:///RoadAccidents.db
Done.


Urban_or_Rural_Area,cy_casualties
Rural,74486
Urban,121251


### 2. Percentage

In [91]:
%%sql
 SELECT Urban_or_Rural_Area, SUM(Number_of_Casualties) * 100 /
 (SELECT SUM(Number_of_Casualties) FROM road_accident WHERE Year = '2022')
 AS percentage
 FROM road_accident
 WHERE Year = '2022'
 GROUP BY Urban_or_Rural_Area

 * sqlite:///RoadAccidents.db


Done.


Urban_or_Rural_Area,percentage
Rural,38
Urban,61


## Casualties by lighting condition

### 1. Sum

In [92]:
%%sql
SELECT
	CASE
		WHEN Light_Conditions IN ('Darkness - lighting unknown', 'Darkness - lights lit', 'Darkness - lights unlit', 'Darkness - no lighting') THEN 'Dark'
		ELSE 'Daylight'
	END AS light_conditions,
	SUM(Number_of_Casualties) AS cy_casualties
 FROM road_accident
 WHERE Year = '2022'
 GROUP BY
 	CASE
		WHEN Light_Conditions IN ('Darkness - lighting unknown', 'Darkness - lights lit', 'Darkness - lights unlit', 'Darkness - no lighting') THEN 'Dark'
		ELSE 'Daylight'
	END

 * sqlite:///RoadAccidents.db
Done.


light_conditions,cy_casualties
Dark,51198
Daylight,144539


### 2. Percentage

In [93]:
%%sql
SELECT
	CASE
		WHEN Light_Conditions IN ('Darkness - lighting unknown', 'Darkness - lights lit', 'Darkness - lights unlit', 'Darkness - no lighting') THEN 'Dark'
		ELSE 'Daylight'
	END AS light_conditions,
	CAST(CAST(SUM(Number_of_Casualties) AS DECIMAL(10,2)) * 100 /
 (SELECT CAST(SUM(Number_of_Casualties) AS DECIMAL(10,2)) FROM road_accident WHERE Year = '2022') AS DECIMAL(10,2))
 AS percentage
 FROM road_accident
 WHERE Year = '2022'
 GROUP BY
 	CASE
		WHEN Light_Conditions IN ('Darkness - lighting unknown', 'Darkness - lights lit', 'Darkness - lights unlit', 'Darkness - no lighting') THEN 'Dark'
		ELSE 'Daylight'
	END

 * sqlite:///RoadAccidents.db
Done.


light_conditions,percentage
Dark,26
Daylight,73


## TOP 10 Casualties by District

In [96]:
%%sql
SELECT Local_Authority_District, SUM(Number_of_Casualties) AS cy_casualties
 FROM road_accident
 WHERE Year = '2022' 
 GROUP BY Local_Authority_District
 ORDER BY cy_casualties DESC
 LIMIT 10


 * sqlite:///RoadAccidents.db
Done.


Local_Authority_District,cy_casualties
Birmingham,4092
Leeds,2764
Cornwall,2092
Bradford,2089
Liverpool,2077
Manchester,1962
Sheffield,1764
County Durham,1708
Cheshire East,1656
Kirklees,1614


## Casualties by Road Surface

In [97]:
%%sql
 SELECT
	CASE
		WHEN Road_Surface_Conditions IN ('Flood over 3cm. deep', 'Wet or damp') THEN 'Wet'
		WHEN Road_Surface_Conditions IN ('Frost or ice', 'Snow') THEN 'Snow/Ice'
		ELSE 'Dry'
	END AS Road_Surface_Conditions,
	SUM(Number_of_Casualties) AS total_casualties
 FROM road_accident
 GROUP BY
	CASE
		WHEN Road_Surface_Conditions IN ('Flood over 3cm. deep', 'Wet or damp') THEN 'Wet'
		WHEN Road_Surface_Conditions IN ('Frost or ice', 'Snow') THEN 'Snow/Ice'
		ELSE 'Dry'
	END


 * sqlite:///RoadAccidents.db
Done.


Road_Surface_Conditions,total_casualties
Dry,279445
Snow/Ice,22781
Wet,115657
